In [2]:
# Check your current folder:
import filepath 
local_folder = filepath.output_folder 
print(local_folder)

/Users/liliguo/Desktop/SMS-NewData/


In [41]:

import infrastructure 
import pygsheets
import time
import pandas as pd 
import numpy as np 
import datetime as dt 
import send_email 


# import publisher information 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
rxmgref = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
publisher  = rxmgref.worksheet('title','Publisher Configurations').get_as_df()

# import offer sheet 
offer_sms = infrastructure.get_smartsheet('offers_sms')
offer_sms = offer_sms[offer_sms['Hitpath ID'].isna() == False]
# import offer sheet from email 
offer_email_raw_data=infrastructure.get_smartsheet('offers_email')
offer_email=offer_email_raw_data[['Hitpath Offer ID','Scheduling Name','Operational Status','Payout']]
offer_email_raw_data = offer_email_raw_data.add_suffix(' from email')

# import mamba 
mamba = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=534096291') 
new_mamba  = mamba.worksheet('title','New Mamba')
schedule = new_mamba.get_as_df()

# define today 
today = pd.to_datetime(dt.datetime.today())

# get holiday restriction list 
holiday_dict = {}
def holiday_restriction(url): 
    holiday_restriction = gc.open_by_url(url)
    holiday_restriction_sheet  = holiday_restriction.worksheet('title','SMS Drop Restrictions')
    holiday_restriction_df = holiday_restriction_sheet.get_as_df()
    holiday_restriction_df.columns = holiday_restriction_df.iloc[3]
    holiday_restriction_df1= holiday_restriction_df.iloc[4:]
    temp = holiday_restriction_df1.set_index('HitPath ID').iloc[:,4:]
    for i in temp.columns:
        df_for_date = temp[i]
        df_for_date1 = df_for_date[df_for_date.str.upper()=='NO DROPS']
        i = pd.to_datetime(i)
        holiday_dict[i] = df_for_date1.index.tolist()
    return holiday_dict
holiday_dict = holiday_restriction('https://docs.google.com/spreadsheets/d/1RuZloAnwCDWQjn5GuRAyEtLoBtQtmiF1nsdJF-ArXYw/edit#gid=0')
holiday_dict = holiday_restriction('https://docs.google.com/spreadsheets/d/1USZQ_CIpfEV-NDH8K9oVy-hy3cTh4uaD0SuksRCtEq8/edit#gid=1068286208')



# La nina 
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1060654066')
lanina_sheet =  lanina.worksheet('title','La Nina (Current)')
lanina_df = lanina_sheet.get_as_df()

# DA payout 
dapayout = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iUxuhBWag0Pamg-GVLpfiJo-ENc2AQzdw3uYhAXVEcQ/edit#gid=1010710644')
offer9287 =  dapayout.worksheet('title','Endurance').get_as_df()
offer9287.columns = offer9287.iloc[0].reset_index()[0].tolist()
offer9287 = offer9287[1:]
offer7878 =  dapayout.worksheet('title','ADT').get_as_df()
offer7878.columns = offer7878.iloc[0].reset_index()[0].tolist()
offer7878 = offer7878[1:]


/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/opt/anaconda3/lib/python3.8/site-packages/p

In [3]:
# check whether there's any duplicated  CCID in La Nina 
lanina_df[lanina_df['Reporting Content ID'].duplicated()]['Reporting Content ID'].tolist()

['', '', '', '', '', '', '', '', '', '']

# Update in La Nina

In [4]:
offer_sms_merge = offer_sms[['Hitpath ID','Status','Vertical']].drop_duplicates()
lanina1  = lanina_df.merge(offer_sms_merge[['Hitpath ID','Status','Vertical']], how = 'left',  left_on = 'OfferIDs',right_on ='Hitpath ID',copy = False )
lanina1
lanina1['Status'] = lanina1['Status'].fillna('')
offer_status_list = ['Offer Status'] +  lanina1['Status'].tolist()
lanina_sheet.update_col(21, offer_status_list)
lanina1['Vertical'] = lanina1['Vertical'].fillna('')
offer_vertical_list = ['Offer Vertical'] +  lanina1['Vertical'].tolist()
lanina_sheet.update_col(22, offer_vertical_list)
# Define the condition and new value
condition_column_index = 4 # Index of the column containing the condition
condition_value = 'Approved - Pipeline Testing'  # Condition to match in the column
condition_column_index_2 = 21
condition_value_2 = 'Passed Test - In Production'
target_column_index = 4 # Index of the column to update
new_value = 'Approved - Passed'  # New value to update


# Iterate over dataframe
for index, row in lanina1.iterrows():
    # Check your condition
    if (row['Content Approval Status'].strip() == 'Pending') & ((row['Status'] == condition_value_2) |(row['Status'] == 'Passed Test - Dupe Bucket')  ):
        # access Data using 
        cell = lanina_sheet.cell((index+2, 4))
        cell.value = 'Approved - Passed' 
        cell.update()


In [5]:
rxpcontent_sheet = lanina.worksheet('title','La Nina for RXP ETL')
rxpcontent = lanina_sheet.get_as_df()

#
flc_url = "https://cf.freedomlender.co/"
mbc_url = "https://cf.mybenefitsclub.com/"
dss_url = "https://members.dollar-sensei.com/"
hzb_url = "https://cf.horizonbenefits.org/"
uaa_url = "https://members.unitedamericanalliance.com/"
svt_url = "https://members.solvent.tools/"
prc_url = "https://members.parentsresourceconnection.co/"
rxp_network_offer_id = pd.read_csv("/Users/liliguo/Desktop/SMS offer id & network offer id.csv")
rxp_network_offer_id['Hitpath ID'] = rxp_network_offer_id['meta_data'].str.extract( '(\d+)')
rxp_network_offer_id['network_offer_id'] = rxp_network_offer_id['network_offer_id'].str.extract( '(\d+)')


# change int to str 
rxpcontent['OfferIDs']  = rxpcontent['OfferIDs'].astype(str).str.replace('.0','')
rxp_network_offer_id['network_offer_id']  = rxp_network_offer_id['network_offer_id'].astype(str).str.replace('.0','')
rxp_network_offer_id['Hitpath ID'] = rxp_network_offer_id['Hitpath ID'].astype(str).str.replace('.0','')

# merge table 
rxpcontent = rxpcontent.merge(rxp_network_offer_id[['Hitpath ID','network_offer_id']] , how = 'left', left_on = 'OfferIDs', right_on = 'Hitpath ID',copy = False)
# production offer wasn't shown in the rxp 
#rxpcontent['network_offer_id'] = rxpcontent['network_offer_id'].fillna('')
rxpcontent.loc[rxpcontent['Hitpath ID'].isna() == False, 'network_offer_id' ] = rxpcontent['network_offer_id'].astype(str).str.replace('nan','')
rxpcontent['Hitpath ID'] = rxpcontent['Hitpath ID'].fillna('')
# offer we couldn't find in rxp
off_hit = offer_sms['Hitpath ID'].astype(str).str.split(".",expand=True)[0].tolist()
print(rxpcontent[(rxpcontent['network_offer_id'].isna())&(rxpcontent['Offer Status'].isin(['Passed Test - In Production','Passed Test - Dupe Bucket'])) & (rxpcontent['OfferIDs'].isin(off_hit)) ]['OfferIDs'].unique())
rest_url =  "o/?cfid="+rxpcontent['OfferIDs'] + "&oid="+rxpcontent['network_offer_id'] +"&emailaddress={{emailaddress}}&firstname={{firstname}}&lastname={{lastname}}&phonenumbermobile={{phonenumbermobile}}&cityname={{cityname}}&postalzipcode={{postalzipcode}}"



rxpcontent['Offer Target URL']  = ''
rxpcontent.loc[(rxpcontent['Type'] == 'FLC')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = flc_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'MBC')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = mbc_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'DSS')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = dss_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'HZB')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = hzb_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'UAA')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = uaa_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'SVT')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = svt_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'PRC')& (rxpcontent['Channel'] == 'TF') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = prc_url + rest_url




['11891']


In [6]:
#rxpcontent['Offer Target URL'].unique()
pattern = r'({{.*?}})'
rxpcontent['Content_rxp'] = rxpcontent['Content']
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|First_name\|.*?}}','{{First_name}}',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|first_name\|.*?}}','{{first_name}}',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|city\|.*?}}','{{city}}',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|region\|.*?}}','{{region}}',regex = True)
#rxpcontent['Content'].str.extractall(pattern)[0].unique()
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{region}}|{{Region}}','%%provincestatename%%',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{first_name}}|{{First_name}}','%%first_name%%',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{today_month}}','%%date:F|today%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{today_dd_mm_yy}}','%%date:d-m-y|today%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{city}}','%%cityname%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{today_mm_dd_yy}}','%%date:m-d-y|today%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{tomorrow_month}}','%%date:F|tomorrow%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{tomorrow_mon_dd_yy}}','%%date:M, d Y|tomorrow%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{random|Erica|Ally|Tom|Grant|Mark|Julie}}','Tom')

# replace the stop to end with ""
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('STOP to end','')
rxpcontent['Content_rxp'].str.extractall(pattern)[0].unique()

rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].astype('str')
rxpcontent['Offer Target URL'] = rxpcontent['Offer Target URL'].fillna('')
rxpcontent['Offer Target URL'] = rxpcontent['Offer Target URL'].astype('str')
# Function to replace the text
def replace_target_url(row):
    if (row['Offer Target URL'] != '') & (row['Offer Target URL'] != np.nan):
        return row['Content_rxp'].replace('{{offer_target_url}}', row['Offer Target URL'])
    return ''

    
rxpcontent['Content_rxp_with_url'] = rxpcontent.apply(replace_target_url, axis=1)
rxpcontent = rxpcontent.drop(columns = ['Drop Count','Content_rxp'])
rxpcontent = rxpcontent[['index', 'Reporting Content ID', 'Long Code Content ID',
       'Content Approval Status', 'Content Status in Platform', 'Type (Pitch)',
       'OfferIDs', 'Advertiser', 'OfferAbbrv', 'Channel', 'Platform', 'Type',
       'Allocation Period (Date Added)', 'Piece Number', 'Copywriter',
       'Date Restrictions', 'Character Count', 'Content','Content_rxp_with_url', 'Review by SMS Team',
       'Link & "STOP to end"', 'Offer Status', 'Offer Vertical', 'Hitpath ID',
       'network_offer_id', 'Offer Target URL']]
rxpcontent_sheet.set_dataframe(rxpcontent, start='A1')


# Update Offer Status in SMS Jump Page Content Review Google Sheet 

In [7]:
"""
smsjumppage = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e8p1LMr4-StJWTF0BK0cwQzSwNLsVSFeRUD1h5pMwfA/edit#gid=1132279237')
smsjumppage_sheet = smsjumppage.worksheet('title','offers-cf')
smsjumppage_df = smsjumppage_sheet.get_as_df()
smsjumppage_df['sid'] = smsjumppage_df['sid'].astype('str').str.split('.',expand = True)[0]
smsjumppage_df1 = smsjumppage_df.merge(offer_sms[['Hitpath ID','Status']], how = 'left',  left_on = 'sid',right_on ='Hitpath ID',copy = False )

smsjumppage_df1['Status'] = smsjumppage_df1['Status'].fillna('')
offer_status_list_jumppage  = ['Offer Status'] +  smsjumppage_df1['Status'].tolist()
smsjumppage_sheet.update_col(2, offer_status_list_jumppage)
"""

"\nsmsjumppage = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e8p1LMr4-StJWTF0BK0cwQzSwNLsVSFeRUD1h5pMwfA/edit#gid=1132279237')\nsmsjumppage_sheet = smsjumppage.worksheet('title','offers-cf')\nsmsjumppage_df = smsjumppage_sheet.get_as_df()\nsmsjumppage_df['sid'] = smsjumppage_df['sid'].astype('str').str.split('.',expand = True)[0]\nsmsjumppage_df1 = smsjumppage_df.merge(offer_sms[['Hitpath ID','Status']], how = 'left',  left_on = 'sid',right_on ='Hitpath ID',copy = False )\n\nsmsjumppage_df1['Status'] = smsjumppage_df1['Status'].fillna('')\noffer_status_list_jumppage  = ['Offer Status'] +  smsjumppage_df1['Status'].tolist()\nsmsjumppage_sheet.update_col(2, offer_status_list_jumppage)\n"

# Upcoming Schedule 

In [8]:
snakes = infrastructure.get_mamba()
snakes['PUBID'] = snakes['Dataset'].str.split('_',expand = True)[2].astype(int)
All_merge = snakes.merge(publisher[['PUBID','Sub Vertical','DMA']], how = 'left', copy = False)
All_merge = All_merge[(All_merge['Date'] > today) ]
All_merge = All_merge.sort_values(['Date','DMA','Sub Vertical','PUBID','Shortcode','Drop'])
upcoming_schedule_mamba = All_merge[['Date','DMA','Sub Vertical','PUBID','Dataset','Drop', 'Time', 'Segment ', 'Send Strategy', 'Offer',
       'Creative', 'Job Name']]

upcoming_schedule_mamba.to_csv('upcoming_schdule.csv',index= False)


/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/infrastructure.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


In [9]:
upcomingschedule = mamba.worksheet('title','Upcoming Schedule')
upcomingschedule.clear()
upcomingschedule.set_dataframe(upcoming_schedule_mamba,(1,1))
upcomingschedule.cell('A1').set_text_format('bold', True)
upcomingschedule.cell('B1').set_text_format('bold', True)
upcomingschedule.cell('C1').set_text_format('bold', True)
upcomingschedule.cell('D1').set_text_format('bold', True)
upcomingschedule.cell('E1').set_text_format('bold', True)
upcomingschedule.cell('F1').set_text_format('bold', True)
upcomingschedule.cell('G1').set_text_format('bold', True)
upcomingschedule.cell('H1').set_text_format('bold', True)
upcomingschedule.cell('I1').set_text_format('bold', True)
upcomingschedule.cell('J1').set_text_format('bold', True)
upcomingschedule.cell('K1').set_text_format('bold', True)
upcomingschedule.cell('L1').set_text_format('bold', True)
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.72, 'green': 0.64, 'blue': 0.81}}, ['Personal Loan'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.95, 'green': 0.75, 'blue': 0.41}}, ['Sweepstakes'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.64, 'green': 0.72, 'blue': 0.81}}, ['Rent To Own'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.63, 'green': 0.85, 'blue': 0.73}}, ['Credit Card'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.95, 'green': 0.94, 'blue': 0.64}}, ['Financial'])

In [10]:
same_offer_from_email = offer_sms[['Hitpath ID','Scheduling Name','Email Offer ID','Status','$ Payout','New Payout','Paused Pubs','Special Pub Payouts','Day Parting','Cap', 'Cap Budget','New Cap Budget']].merge(offer_email_raw_data[['Hitpath Offer ID from email','Scheduling Name from email','Operational Status from email','Payout from email','Paused PubIDs from email','Day Restrictions from email','Drop Time Restrictions from email','Budget from email', 'Cap from email']],left_on='Email Offer ID', right_on = 'Hitpath Offer ID from email', how = 'inner')
same_offer_from_email['Hitpath ID'] = same_offer_from_email['Hitpath ID'].astype(str).str.split('.',expand = True)[0]
#offer_sms['Email Offer ID'] = offer_sms['Email Offer ID'].astype(str).str.split('.',expand = True)[0]
email_list = offer_sms['Email Offer ID'].unique().tolist()
email_list = [x for x in email_list if x !='nan']
sms_live_pubid = publisher['PUBID'].astype(str).unique().tolist()
# special payout for 9287 and 7878 
offer9287['PUBID'] = offer9287['PUBID'].astype(str).str.split(".",expand = True)[0]
sp9287 = offer9287.loc[(offer9287['STATUS'] == 'Live') & (offer9287['PUBID'].astype(str).isin(sms_live_pubid)) ,][['PUBID','NEW RATE']].to_string(index=False,header =False)
offer7878['PUBID'] = offer7878['PUBID'].astype(str).str.split(".",expand = True)[0]
sp7878 = offer7878.loc[(offer7878['STATUS'] == 'Live') & (offer7878['PUBID'].astype(str).isin(sms_live_pubid)) ,][['PUBID','NEW RATE']].to_string(index=False,header = False)
if "Empty DataFrame" in sp7878:
    sp7878 = ''
offer = same_offer_from_email[same_offer_from_email['Email Offer ID'].isnull() == False]

# speical payout 
offer['Special Payout from Email'] = ''
offer['Special Pub Payouts'] = offer['Special Pub Payouts'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer.loc[offer['Email Offer ID'] == 9287,'Special Payout from Email' ] = sp9287
offer.loc[offer['Email Offer ID'] == 7878,'Special Payout from Email' ] = sp7878
offer['Special Payout Alert'] = ''
offer.loc[(offer['Special Pub Payouts']!=offer['Special Payout from Email']) , 'Special Payout Alert'] = 'Please update Special Payout'

offer['Paused Alert'] =  ''
offer.loc[(offer['Status'].str.lower().str.contains('set up|pause|pitch|archive',na = False) == False) & (offer['Operational Status from email'].str.contains('Paused') ),'Paused Alert'] = 'Please Pause offer in SMS'

offer['Payout Alert'] = ''
offer['SMS_payout_rate'] = offer['$ Payout'].str.extract(r'([\d\.]+)').astype(float)
offer['SMS_new_payout_rate'] = offer['New Payout'].str.extract(r'([\d\.]+)').astype(float)
offer['Email_payout_rate'] = offer['Payout from email'].str.extract(r'([\d\.]+)').astype(float)
offer['Payout Alert'] = ''
offer.loc[(offer['SMS_payout_rate']!=offer['Email_payout_rate']) & (offer['SMS_new_payout_rate']!=offer['Email_payout_rate'])&(offer['$ Payout']!=offer['Payout from email']), 'Payout Alert'] = 'Please update the payout'

offer['Paused Pubids list'] = offer['Paused PubIDs from email'].str.split('\n|\t|,|"| ')
offer['SMS current Paused Pubids list'] = offer['Paused Pubs'].str.split('\n|\t|,|"| ')
offer['SMS current Paused Pubids list'] = offer['SMS current Paused Pubids list'].apply(lambda x: sorted(x) if isinstance(x, (list, tuple))  else list())
offer['Update Needed in SMS: Paused Pubids'] = offer['Paused Pubids list'].apply(lambda x: sorted(list(set(list(x)) & set(sms_live_pubid))) if isinstance(x, (list, tuple))  else list())
offer['Update Needed in SMS: Paused Pubids'] = offer['Update Needed in SMS: Paused Pubids'].apply(lambda x: [int(i) for i in x if i != ''])
offer['SMS current Paused Pubids list'] = offer['SMS current Paused Pubids list'].apply(lambda x: [int(i) for i in x if i != ''])
offer.loc[offer['Update Needed in SMS: Paused Pubids'] == offer['SMS current Paused Pubids list'],'Update Needed in SMS: Paused Pubids'] = ''
offer['Update Needed in SMS: Paused Pubids'] = offer['Update Needed in SMS: Paused Pubids'].apply(lambda x: '' if x == list() else x) 
offer['Update Needed in SMS: Paused Pubids'] = offer['Update Needed in SMS: Paused Pubids'].apply(lambda x: '\n'.join(map(str, x)))
offer.loc[offer['SMS current Paused Pubids list'].isnull(), 'SMS current Paused Pubids list'] = ''

offer['Day Parting from Email'] = offer['Day Restrictions from email'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x ) + ' ' + offer['Drop Time Restrictions from email'].apply(lambda x: "" if x == 'None'  or pd.isnull(x) else x )
offer['Day Parting'] = offer['Day Parting'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer['Day Parting Alert'] = ''
offer.loc[offer['Day Parting'].str.replace(" ","").str.lower()!=offer['Day Parting from Email'].str.replace(" ","").str.lower(),'Day Parting Alert'] = 'Please update the Day Parting'

offer['Budget Alert'] =''
offer['Cap from email1'] = offer['Cap from email'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer['Cap1'] = offer['Cap'].apply(lambda x: "" if x == 'No Cap'  or pd.isnull(x) else x )
offer['New Cap Budget1'] = offer['New Cap Budget'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer['Budget from email1'] = offer['Budget from email'].apply(lambda x: "" if x == 'None'  or pd.isnull(x) else x )
offer['Cap Budget1'] = offer['Cap Budget'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer.loc[offer['Cap1']!=offer['Cap from email1'],'Budget Alert'] = offer['Budget Alert']+"Please update budget cap. "
offer['Budget from email2'] = offer['Budget from email1'].str.extract(r'(\d+)')
offer.loc[(offer['Budget from email1']!=offer['Cap Budget1'])&(offer['Budget from email2']!=offer['Cap Budget1'])& (offer['Budget from email1']!=offer['New Cap Budget1']) & (offer['Budget from email2']!=offer['New Cap Budget1']), 'Budget Alert'] = offer['Budget Alert']+"Please update budget. "
offer = offer.drop(['SMS_payout_rate','Email_payout_rate','Paused Pubids list','SMS current Paused Pubids list','SMS_new_payout_rate','Cap from email1','Cap1','New Cap Budget1','Budget from email1','Cap Budget1','Budget from email2'],axis = 1)
# if offer is paused, then all alert is ''
offer.loc[(offer['Status'] =='Internal Pause - Archive'), 'Budget Alert'] = ''
offer.loc[(offer['Status'] =='Internal Pause - Archive'), 'Update Needed in SMS: Paused Pubids'] = ''
offer.loc[(offer['Status'] =='Internal Pause - Archive'), 'Paused Alert'] = ''
offer.loc[(offer['Status'] =='Internal Pause - Archive'), 'Payout Alert'] = ''
offer.loc[(offer['Status'] =='Internal Pause - Archive'), 'Day Parting Alert'] = ''
offer.loc[(offer['Status'] =='Internal Pause - Archive'), 'Special Payout Alert'] = ''
#offer[(offer['SMS current Paused Pubids list'] !=offer['Update Needed in SMS: Paused Pubids'])]

# communicte with Daniel to uniform the time format 
offer_upload = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Y0NjJBQBSXhTangk61PrOlKJBNZat9mmZGanAPuLma4/edit#gid=0')
offer_upload_wks  = offer_upload.sheet1
headers = offer_upload_wks.get_row(1)
offer_upload_wks.clear(start='A2')
offer_upload_wks.update_row(1, headers)
offer_upload_wks.set_dataframe(offer,(2,1))

# EDIT EMAIL MESSAGE: 
need_update_offer_list = offer.loc[((offer['Update Needed in SMS: Paused Pubids']!= '') | (offer['Paused Alert']!= '') | (offer['Payout Alert']!= '') |  (offer['Day Parting Alert']!= '') | (offer['Budget Alert']!= '') | (offer['Special Payout Alert']!='')), ]['Hitpath ID' ].unique().tolist()
need_update_offer_list_str =  ','.join(need_update_offer_list)
if len(need_update_offer_list) >0: 
    message = "Hi AM Team, we found the information discrepancy between SMS Offer Sheet and Email Offer Management Smartsheet for the following email offers and please update the information in the SMS Offer Sheet. Thank you!\nHere's the list: \n" + need_update_offer_list_str
    #receipient_list = ['lili@rxmg.com']
    receipient_list =  ['offernotices@rxmg.com']
    for i in receipient_list:
        offer_upload.share(i, role='commenter', type='user', emailMessage=message)


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_1974/3189346744.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer['Special Payout from Email'] = ''
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_1974/3189346744.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer['Special Pub Payouts'] = offer['Special Pub Payouts'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_1974/3189346744.py:21: S

In [43]:
offer_sms['Day Parting'] = offer_sms['Day Parting'].str.upper()
offer_sms['Day Parting'] = offer_sms['Day Parting'].str.replace(' ','')
offer_sms['Day Parting'] = offer_sms['Day Parting'].str.replace('M-F','MONDAY-FRIDAY')
All_merge['Hitpath Offer ID'] = All_merge['Hitpath Offer ID'].astype('str').str.split('.',expand = True)[0]
offer_sms['Hitpath ID'] = offer_sms['Hitpath ID'].astype('str').str.split('.',expand = True)[0]

alert_report = All_merge.merge(offer_sms[['Hitpath ID','Email Offer ID','Scheduling Name','Day Parting','Status','Paused Pubs']], left_on ='Hitpath Offer ID',right_on = 'Hitpath ID', how = 'left') 

offer_email = offer_email[offer_email['Hitpath Offer ID'].isna() == False]
alert_report = alert_report.merge(offer_email[['Hitpath Offer ID','Operational Status']],left_on ='Email Offer ID' ,right_on ='Hitpath Offer ID', how = 'left',copy= False)

alert_report['Alert'] = ''
alert_report.loc[alert_report['Creative'].isna(),'Alert'] = 'No Creative'
alert_report['Weekday'] = pd.to_datetime(alert_report['Date']).dt.day_name()
alert_report.loc[alert_report['Creative'].isna(),'Alert'] = 'No Creative'
alert_report.loc[alert_report['Status'].str.lower().str.contains('set up|pause|pitch|archive',na = False), 'Alert' ] = 'Not Live Offer'
for i in holiday_dict: 
    alert_report.loc[(alert_report['Date'] == i) & (alert_report['Hitpath ID'].isin(holiday_dict[i])), 'Alert' ] = 'Holiday Restriction'
alert_report.loc[(alert_report['Operational Status'].str.lower().str.contains('paused',na = False)),  'Alert'  ] = 'Not Live Offer in Email'
alert_report.loc[(alert_report['Weekday'].isin(['Saturday','Sunday']) ) & (alert_report['Day Parting'].str.contains('MONDAY-FRIDAY',na = False)),'Alert'] = 'Day Restriction'
alert_report['Paused Pubs'] = alert_report['Paused Pubs'].astype('str')
alert_report['PUBID'] = alert_report['PUBID'].astype('str')
alert_report['Paused Alert'] = alert_report.apply(lambda x:x['PUBID'] in x['Paused Pubs'], axis=1)
alert_report.loc[alert_report['Paused Alert']== True,'Alert'] = 'Paused Pubid'

alert_report['Responsible DMA'] = alert_report['DMA']
alert_report.loc[alert_report['Send Strategy'] == 'PT','Responsible DMA' ] = 'Nathan'
alert_report = alert_report[alert_report['Alert'] != ''][['Responsible DMA','Alert','Scheduling Name','Day Parting','Status','Date','Sub Vertical','PUBID','Dataset','Drop', 'Time', 'Segment ', 'Send Strategy', 'Offer',
       'Creative']]
# alert from email offer 
filename = local_folder+'swap_report.csv'
alert_report.to_csv(filename, index = False)


NameError: name 'All_merge' is not defined

In [14]:
overlap_report = pd.read_excel( local_folder+'sms-overlap.xlsx')

overlap_report['pub1_Dataset'] =  overlap_report['pub1_sc'].str.split('-',expand = True)[2]+'_'+overlap_report['pub1_dp_ds']+'_'+ overlap_report['pubid1'].astype('str')
overlap_report['pub2_Dataset'] =  overlap_report['pub2_sc'].str.split('-',expand = True)[2]+'_'+overlap_report['pub2_dp_ds']+'_'+ overlap_report['pubid2'].astype('str')
high_overlap = overlap_report[(overlap_report['pct_overlap'] > 0.1) & (overlap_report['pct_overlap'] <1)] 
high_overlap_schedule = high_overlap[['pub1_Dataset','pub2_Dataset','pct_overlap']].merge(All_merge, how = 'left', left_on = 'pub1_Dataset', right_on = 'Dataset',copy = False)
high_overlap_schedule = high_overlap_schedule.merge(All_merge[['Dataset','Hitpath Offer ID','Date']],how = 'left', left_on = ['pub2_Dataset','Hitpath Offer ID'], right_on = ['Dataset','Hitpath Offer ID'], copy = False )
high_overlap_schedule['Offer Gap between Overlapped Datasets'] = abs(high_overlap_schedule['Date_x'] - high_overlap_schedule['Date_y'])
high_overlap_schedule = high_overlap_schedule.loc[(abs((high_overlap_schedule['Date_x'] - high_overlap_schedule['Date_y']).dt.days)<=2) & ((abs((high_overlap_schedule['Date_x'] - high_overlap_schedule['Date_y']).dt.days)>=0)),]
high_overlap_schedule = high_overlap_schedule.drop(['Dataset_x','Dataset_y'], axis = 1)
high_overlap_schedule = high_overlap_schedule[['Hitpath Offer ID','pub1_Dataset', 'Date_x','DMA','Sub Vertical', 'Send Strategy','pub2_Dataset','Date_y','pct_overlap','Offer Gap between Overlapped Datasets']]
high_overlap_schedule = high_overlap_schedule.rename(columns = {'Date_x':'Date','Date_y':'Overlapped_pub_Date','pub1_Dataset':'Dataset','pub2_Dataset':'Overlapped_pub'})
high_overlap_schedule['Hitpath Offer ID'] = high_overlap_schedule['Hitpath Offer ID'].astype('int').astype('str')
high_overlap_schedule['pct_overlap'] =( high_overlap_schedule['pct_overlap']*100).round(2).astype('str') + '%'
high_overlap_schedule['Warning'] = "Please move the drops in " +high_overlap_schedule['Dataset'] +" with offer "+ high_overlap_schedule['Hitpath Offer ID']+ " to 3 days away from "+ high_overlap_schedule['Overlapped_pub_Date'].astype('str') + " to avoid high overlap"
high_overlap_schedule.to_csv(local_folder+'Overlap_Accounts_Swap_Report.csv', index = False)

In [15]:

toaddr =  ['sms@rxmg.com','offernotices@rxmg.com','prasad@rxmg.com']
filename = [local_folder+'swap_report.csv',local_folder+'Overlap_Accounts_Swap_Report.csv']
no_swap = len(alert_report)
subject_line = f"SMS Team Upcoming Schedule & Swap 'Report & Overlap Accounts Swap Report"
email_body = "Hi Team,\n\nHere's the Upcoming Schedule link: \nhttps://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1189721992 \n\nHere's the swap details from the swap report(attached): \n"
for i in alert_report['Responsible DMA'].unique(): 
    email_body += i +":"+'\n' 
    dma = alert_report[alert_report['Responsible DMA'] == i]
    for i in range(0,len(dma)): 
        row_dma = dma.iloc[i]
        dma_send_strategy = row_dma['Send Strategy']
        dma_date =row_dma['Date'].strftime('%Y-%m-%d')
        dma_dataset = row_dma['Dataset']
        dma_alert = row_dma['Alert']
        email_body += f"  The {dma_send_strategy} drop on {dma_date} in {dma_dataset} need to be swapped/updated due to {dma_alert}\n"
email_body += "\n\nHere's the swap details from the Overlap Accounts Swap Report(attached) due to less than 3 days' offer gap between highly overlapped datasets: \n"
for i in high_overlap_schedule['DMA'].unique():
    email_body += i +":"+'\n' 
    dma = high_overlap_schedule[high_overlap_schedule['DMA'] == i] 
    for i in range(0,len(dma)):
        row_dma = dma.iloc[i]
        warning = row_dma['Warning']
        
        email_body += f"  {warning}\n"

email_body += '\n\nHave a nice day!\n\nThanks\nLili Guo\n\n'

for i in toaddr:
    send_email.send_email(filename,subject_line,email_body,i)



connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP g22-20020a1709029f9600b001c61afa7009sm1976028plq.114 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP g22-20020a1709029f9600b001c61afa7009sm1976028plq.114 - gsmtp'
connect: b'smtp.gmail.com ESMTP g22-20020a1709029f9600b001c61afa7009sm1976028plq.114 - gsmtp'
send: 'ehlo 1.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.ip6.arpa\r\n'
reply: b'250-smtp.gmail.com at your service, [47.145.216.112]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [47.145.216.112]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
sen